### Demo #7 Using Multiagent Frameworks for Situation Reports (SITREP)
Use a multiagent framework to create a routine newsletter or situation report (SITREP). The idea is that you can take all the bits and pieces from the previous demos and deploy an army of agents to gather internal/external data, analyze data, and synthesize findings.

There are several frameworks you can utilize to do this like crewai (see Demo #6) and Autogen (https://microsoft.github.io/autogen/). In this demo, we'll be using Autogen.

Note, datasets used by the agents are summarized datasets from the MER Structured Dataset (MSD). I summarize the dataset beforehand to reduce the instructions (and possibly errors) I need to give to the agent. 

In [76]:
pip install autogen pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [77]:
load_dotenv()

True

Create a load_data function that can be used to import the datasets.

https://microsoft.github.io/autogen/0.2/docs/topics/code-execution/user-defined-functions#define-the-function

In [78]:
import os
from dotenv import load_dotenv
import autogen
from autogen.coding.func_with_reqs import with_requirements
import pandas

In [79]:
@with_requirements(python_packages=["pandas"], global_imports=["pandas"])
def load_data() -> pandas.DataFrame:
    """Load achievement data.

    Returns:
        pandas.DataFrame: A DataFrame with the following columns: operatingunit, psnu, funding_agency, prime_partner_name, age_2019, sex, fiscal_year, indicator, cumulative, targets
    """
    data = pandas.read_csv("prevention_achievements.csv")
    
    return pandas.DataFrame(data)

In [80]:
from pathlib import Path
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

work_dir  = Path("Datasets")
os.listdir(work_dir) # Check that it works

executor = LocalCommandLineCodeExecutor(work_dir=work_dir, functions=[load_data])

In [81]:
nlnl = "\n\n"
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

# Add on the new functions
code_writer_system_message += executor.format_functions_for_prompt()

print(code_writer_system_message)


You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or pe

LLM configuration

https://microsoft.github.io/autogen/0.2/docs/topics/llm_configuration

In [82]:
llm_config = { "config_list": [
  {
    "model": os.environ["AZURE_OPENAI_DEPLOYMENT"],
    "api_type": "azure",
    "api_key": os.environ["AZURE_OPENAI_KEY"],
    "base_url": os.environ["AZURE_OPENAI_ENDPOINT"],
    "api_version": os.environ["AZURE_OPENAI_VERSION"]
  }
] }

# assistant = AssistantAgent("assistant", llm_config=llm_config)
# user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# # Start the chat
# user_proxy.initiate_chat(
#     assistant,
#     message="Tell me a joke about NVDA and TESLA stock prices.",
# )

In [117]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="NEVER",
)

code_writer_agent = autogen.ConversableAgent(
    "Coder",
    system_message=code_writer_system_message,
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
)

code_executor_agent = autogen.ConversableAgent(
    name="Executor",
    llm_config=False,
    system_message="Executor. Execute the code written by the Coder and report the result.",
    code_execution_config={
        "executor": executor,
    },
    human_input_mode="NEVER",
)

data_analyst = autogen.AssistantAgent(
    name="Analyst",
    llm_config=llm_config,
    system_message="""Analyst.
    You analyze summarized datasets generating by the Executor and figure out the main message (i.e., key achievement gaps and areas of concern).
    
    Ignore any results that are inconclusive (i.e. NA, NaN, Inf, etc.).
    """
)

sitrep_editor = autogen.AssistantAgent(
    name="Editor-in-Chief",
    system_message="""Editor-in_Chief.
    Find and format key messages from the summarized datasets in bullet point form to deliver to decision-makers in the form of a newsletter.
    Summarize the key messages for each operatingunit in sections.
    Receive any feedback from the Critic and reengage with the other agents to refine the key messages.

    Remove any results that are inconclusive (i.e. NA, NaN, Inf, etc.)
    Do not include any bullet points describing the code or methods used to summarize the data.
""",
    llm_config=llm_config,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check claims and code from other agents and provide feedback. Evaluate the quality of the analysis and the clarity of the message.",
    llm_config=llm_config,
)

Set up the team!

In [118]:
groupchat = autogen.GroupChat(agents=[user_proxy, code_writer_agent, code_executor_agent, sitrep_editor], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

Start chat - test the code agents

In [85]:
# chat_result = code_executor_agent.initiate_chat(
#     code_writer_agent,
#     message="Please use the load_data function to load the data and please calculate the average targets for all the operatingunits.",
#     summary_method="reflection_with_llm",
# )

# print(chat_result.summary)

In [122]:
chat_result2 = user_proxy.initiate_chat(
    manager,
    message=""" 
    Please use the load_data function to load the data then conduct a data analysis of the achievements for 
    each geographic group: "operatingunit" (i.e. country or region), "psnu" (i.e. the subregion of the operatingunit); 
    each population group: "age_2019" (i.e. the age) and "sex" (i.e. the sex). 

    All the rows are for each specific indicator indicated in the "indicator" column.
    To calculate the "indicator achievement", you will have to summarize (i.e. groupby and sum targets, cumulative) then divide the "cumulative" value by the "target" value. 
    For each operatingunit, calculate the indicator achievement for every combination of geographic and population categories.
    Express achievement as percentages.

    Create a newsletter to PEPFAR stakeholders of relevant findings and insights for each operatingunit of the data. 
    There should be one section for each operatingunit with the following information:  
    - The operatingunit name
    - The indicator
    - Try to create a sucinct narratives for 
        - which psnu and population combinations have 0 percent achievement.
        - which psnu and population combinations have below 50 percent achievement.
        - provide detailed summary at the end stating which area and which population(s) we need to focus on. Try to group things as much as possible. Group the population if the make sense. Example, psnu "a" has below 0 percent achievement for the under 24 age groups.
    """,
    summary_method="reflection_with_llm",
)

print(chat_result2.summary)

Admin (to chat_manager):

 
    Please use the load_data function to load the data then conduct a data analysis of the achievements for 
    each geographic group: "operatingunit" (i.e. country or region), "psnu" (i.e. the subregion of the operatingunit); 
    each population group: "age_2019" (i.e. the age) and "sex" (i.e. the sex). 

    All the rows are for each specific indicator indicated in the "indicator" column.
    To calculate the "indicator achievement", you will have to summarize (i.e. groupby and sum targets, cumulative) then divide the "cumulative" value by the "target" value. 
    For each operatingunit, calculate the indicator achievement for every combination of geographic and population categories.
    Express achievement as percentages.

    Create a newsletter to PEPFAR stakeholders of relevant findings and insights for each operatingunit of the data. 
    There should be one section for each operatingunit with the following information:  
    - The operatingunit na